## Board generator

In [94]:
import json

In [95]:
hw_ratio = 960/800 # height/width

u = 15.8   # 1u is the length of 5 steps
ax = u/4*hw_ratio  # 1a is the distance between (the centers of) two steps
ay = u/4

color_dict = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5'}

# draw the board by walking to one step after another and choosing the direction where the next step will be
class Stepper:
    def __init__(self, x, y):
        # supply initial coordinates from where the steps are taken
        self.x = x
        self.y = y
        self.id = 0
        
    def set_coords(self, x, y):
        self.x = x
        self.y = y
        
    def get_coords(self):
        # get the coordinates and do not increment the id
        return (self.x, self.y)
        
    def get_step(self):
        # create a step and increment the id
        s = {'id': self.id, 'x': f'{self.x:.2f}%', 'y': f'{self.y:.2f}%'}
        self.id += 1
        return s
    
    def get_outer(self, color_id):
        # create coordinates for the circles around the steps
        return {'x': f'{self.x:.2f}%', 'y': f'{self.y:.2f}%', 'color': color_dict[color_id]}
    
    def get_house(self, color_id):
        s = {'id': 1000+self.id, 'x': f'{self.x:.2f}%', 'y': f'{self.y:.2f}%', 'color': color_dict[color_id]}
        self.id += 1
        return s
    
    def get_home(self, color_id):
        s = {'id': -self.id, 'x': f'{self.x:.2f}%', 'y': f'{self.y:.2f}%', 'color': color_dict[color_id]}
        self.id += 1
        return s
        
    def n(self, scale=1):
        self.y -= ay*scale
    
    def ne(self, scale=1):
        self.x += 0.7*ax*scale
        self.y -= 0.7*ay*scale
        
    def e(self, scale=1):
        self.x += ax*scale
        
    def se(self, scale=1):
        self.x += 0.7*ax*scale
        self.y += 0.7*ay*scale
        
    def s(self, scale=1):
        self.y += ay*scale
        
    def sw(self, scale=1):
        self.x -= 0.7*ax*scale
        self.y += 0.7*ay*scale
        
    def w(self, scale=1):
        self.x -= ax*scale
        
    def nw(self, scale=1):
        self.x -= 0.7*ax*scale
        self.y -= 0.7*ay*scale

In [96]:
# Generate the steps

step = Stepper(50+1.2*u*hw_ratio, 50-2.9*u)

allsteps = []

def do4(f):
    for i in range(4):
        allsteps.append(step.get_step())
        f()

base1 = step.get_coords()

do4(step.sw)
do4(step.w)
do4(step.nw)
do4(step.sw)

base2 = step.get_coords()

do4(step.se)
do4(step.s)
do4(step.w)
do4(step.s)

base3 = step.get_coords()

do4(step.e)
do4(step.s)
do4(step.sw)
do4(step.se)

base4 = step.get_coords()

do4(step.ne)
do4(step.e)
do4(step.se)
do4(step.ne)

base5 = step.get_coords()

do4(step.nw)
do4(step.n)
do4(step.e)
do4(step.n)

base6 = step.get_coords()

do4(step.w)
do4(step.n)
do4(step.ne)
do4(step.nw)


In [97]:
outer = []    # circles around homes and houses and the base step
homes = []
houses = []

# Generate the homes
step = Stepper(0, 0)

def do4(f, color_id):
    f(scale=1.4)
    homes.append(step.get_home(color_id))
    outer.append(step.get_outer(color_id))
    for i in range(3):
        f()
        homes.append(step.get_home(color_id))
        outer.append(step.get_outer(color_id))

homebuilding = [(base1, step.w, 0), (base2, step.s, 1), (base3, step.s, 2),
                (base4, step.e, 3), (base5, step.n, 4), (base6, step.n, 5)]

for base, direction, color_id in homebuilding:
    step.set_coords(*base)
    do4(direction, color_id)

# Generate the houses
step = Stepper(0, 0)


def draw_house(f1, f2, color_id):
    f1(scale=1.4)
    houses.append(step.get_house(color_id))
    outer.append(step.get_outer(color_id))
    f1()
    houses.append(step.get_house(color_id))
    outer.append(step.get_outer(color_id))
    for i in range(2):
        f2()
        houses.append(step.get_house(color_id))
        outer.append(step.get_outer(color_id))
        

housebuilding = [(base1, step.s, step.sw, 0), (base2, step.e, step.se, 1), (base3, step.ne, step.e, 2),
                 (base4, step.n, step.ne, 3), (base5, step.w, step.nw, 4), (base6, step.sw, step.w, 5)]

for base, dir1, dir2, color_id in housebuilding:
    step.set_coords(*base)
    draw_house(dir1, dir2, color_id)

In [99]:
# data dictionary that will be saved as a json file
data = {'steps': allsteps, 'outer': outer, 'homes': homes, 'houses': houses}

with open('src/components/board/boarddata6.json', 'w') as fp:
    json.dump(data, fp)